# Import StreamPoseML

In [ ]:
%%capture
%pip install stream_pose_ml

# Set input and output directories

In [ ]:
import os
import time

# Inputs
example_input_directory = "../../example_data/input"
example_output_directory = f"../../example_data/output-{time.time_ns()}"

source_annotations_directory = os.path.join(example_input_directory, "source_annotations")
source_videos_directory = os.path.join(example_input_directory, "source_videos")

# Outputs

# The location to output sequence data
sequence_data_directory = os.path.join(example_output_directory, "sequences")

# The location to output keypoint data
keypoints_data_directory = os.path.join(example_output_directory, "keypoints")

# The location to output datasets
merged_annotation_output_directory = os.path.join(example_output_directory, "datasets")

# The place to save trained models
trained_models_output_directory = os.path.join(example_output_directory, "trained_models")


for dir in [sequence_data_directory, keypoints_data_directory, merged_annotation_output_directory, trained_models_output_directory]:
    os.makedirs(dir, exist_ok=True)

# Generate Keypoints and Sequence Data

In [ ]:

import stream_pose_ml.jobs.process_videos_job as pv

folder = f"run-preproccessed-{time.time_ns()}"  
keypoints_path = f"{keypoints_data_directory}/{folder}"
sequence_path = f"{sequence_data_directory}/{folder}"

data = pv.ProcessVideosJob().process_videos(
    src_videos_path=source_videos_directory,
    output_keypoints_data_path=keypoints_path,
    output_sequence_data_path=sequence_path,
    write_keypoints_to_file=True,
    write_serialized_sequence_to_file=True,
    limit=None,
    configuration={},
    preprocess_video=True,
    return_output=False
)

print(f"Generated keypoints are located at {data['keypoints_path']}")
print(f"Generated sequences are located at {data['sequence_path']}")

# Merge video sequence data into a dataset

In [ ]:
import stream_pose_ml.jobs.build_and_format_dataset_job as data_builder 

db = data_builder.BuildAndFormatDatasetJob()

dataset_file_name = "preprocessed_flatten_on_example_10_frames_5"

dataset = db.build_dataset_from_data_files(
    annotations_data_directory=source_annotations_directory,
    sequence_data_directory=sequence_data_directory,
    limit=None,
)

formatted_dataset = db.format_dataset(
    dataset=dataset,
    pool_frame_data_by_clip=False,
    decimal_precision=4,
    include_unlabeled_data=True,
    include_angles=True,
    include_distances=True,
    include_normalized=True,
    segmentation_strategy="flatten_into_columns",
    segmentation_splitter_label="step_type",
    segmentation_window=10,
    segmentation_window_label="weight_transfer_type",
)

db.write_dataset_to_csv(
    csv_location=merged_annotation_output_directory,
    formatted_dataset=formatted_dataset,
    filename=dataset_file_name
)


# Train a model (Gradient Boost)

### Create some synthetic data from our limited sample data for training demo purposes...

In [ ]:
# Our dataset example is comically small, so let's make it bigger for the sake of a training demonstration

import pandas as pd
data_file = os.path.join(merged_annotation_output_directory, f"{dataset_file_name}.csv")
data_file_expanded = os.path.join(merged_annotation_output_directory, f"{dataset_file_name}-EXPANDED.csv")

df = pd.read_csv(data_file)
df = pd.concat( map(pd.read_csv, [data_file for _ in range(100)]), ignore_index=True) 
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv(data_file_expanded)

display(len(df))
display(df.head())


In [ ]:
# NOTE This example will be very overfit!
from stream_pose_ml.learning import model_builder as mb

# Mapping string categories to numerical
value_map = {
    "weight_transfer_type": {
        "Failure Weight Transfer": 0,
        "Successful Weight Transfer": 1,
    },
    "step_type": {
        "Left Step": 0,
        "Right Step": 1,
    },
}
# Columns we know we'll always want to drop
drop_list = ["video_id", "step_frame_id", "frame_number", "step_type"]
# Only keep these columns (including target)
column_whitelist = []


model_builder = mb.ModelBuilder()

model_builder.load_and_prep_dataset_from_csv(
    path=data_file_expanded,
    target="weight_transfer_type",
    value_map=value_map,
    column_whitelist=column_whitelist,
    drop_list=drop_list,
)

model_builder.set_train_test_split(
    balance_off_target=True,
    upsample_minority=True,
    downsample_majority=False,
    use_SMOTE=False,
    random_state=40002,
)
model_builder.train_gradient_boost()
model_builder.evaluate_model()

# Train a model (Random Forest)

In [ ]:
from random import randint
# NOTE This example will be very overfit!
from stream_pose_ml.learning import model_builder as mb

# Mapping string categories to numerical
value_map = {
    "weight_transfer_type": {
        "Failure Weight Transfer": 0,
        "Successful Weight Transfer": 1,
    },
    "step_type": {
        "Left Step": 0,
        "Right Step": 1,
    },
}
# Columns we know we'll always want to drop
drop_list = ["video_id", "step_frame_id", "frame_number", "step_type"]
# Only keep these columns (including target)
column_whitelist = []

model_builder.load_and_prep_dataset_from_csv(
    path=data_file_expanded,
    target="weight_transfer_type",
    value_map=value_map,
    column_whitelist=column_whitelist,
    drop_list=drop_list,
)

model_builder.set_train_test_split(
    balance_off_target=True,
    upsample_minority=True,
    downsample_majority=False,
    use_SMOTE=False,
    random_state=40002,
)
# model_builder.run_pca(num_components=200)

param_dist = {
    "n_estimators": [randint(400, 600)],
    "max_depth": [randint(9, 15)],
    "max_features": [randint(4, 12)],
}
rf_params = {
    "n_estimators": [20,50,100,200],
    "max_depth": 9,
    "max_leaf_nodes": 63,
}

model_builder.train_random_forest(
    use_random_search=True, 
    params=rf_params, 
    param_dist=param_dist, 
    iterations=50, 
    random_state = 123
)
model_builder.evaluate_model()

# Save model for use in the Web Application

This will ouput a pickled dictionary with the model saved to the "classifier" key.


In [ ]:
""" WRITE NOTES ON THIS RUN HERE """
notes = """
Gradient Boost classifier (90% ROC AUC) trained on dataset preprocessed_flatten_on_example_10_frames, a 10 frame window with flat column 2d angles + distances, and randomly upsampled
    """

model_builder.save_model_and_datasets(notes=notes, model_type="gradient-boost", model_path=trained_models_output_directory)

# Other kinds of datasets from sequences

### Dataset with raw x, y, z joint data, with one frame per row

In [ ]:
sequence_data_directory = "../../data/sequences/run-preproccessed-1680117203184086000"
db = data_builder.BuildAndFormatDatasetJob()
dataset = db.build_dataset_from_data_files(
    annotations_data_directory=source_annotations_directory,
    sequence_data_directory=sequence_data_directory,
    limit=None,
)

formatted_dataset = db.format_dataset(
    dataset=dataset,
    pool_frame_data_by_clip=False,
    decimal_precision=4,
    include_unlabeled_data=True,
    include_joints=True,
    include_z_axis=True,
    include_angles=False,
    include_distances=False,
    include_normalized=False,
    segmentation_strategy="none",
)

db.write_dataset_to_csv(
    csv_location=merged_annotation_output_directory,
    formatted_dataset=formatted_dataset,
    filename="preprocessed_frame_joint_data"
)

### Split on Step Type, pooled temporal dynamics with angles and distances, only the last 10 frames

In [ ]:
db = data_builder.BuildAndFormatDatasetJob()
dataset = db.build_dataset_from_data_files(
    annotations_data_directory=source_annotations_directory,
    sequence_data_directory=sequence_data_directory,
    limit=None,
)

formatted_dataset = db.format_dataset(
    dataset=dataset,
    pool_frame_data_by_clip=True,
    decimal_precision=4,
    include_unlabeled_data=True,
    include_angles=True,
    include_distances=True,
    include_normalized=True,
    segmentation_strategy="split_on_label",
    segmentation_splitter_label="step_type",
    segmentation_window=10,
    segmentation_window_label="weight_transfer_type",
)

db.write_dataset_to_csv(
    csv_location=merged_annotation_output_directory,
    formatted_dataset=formatted_dataset,
    filename="pooled_angles_distances_last_10_frames"
)

### Flatten columns over 10 frame window on step type (arbitrary start / end)

In [ ]:
db = data_builder.BuildAndFormatDatasetJob()
dataset = db.build_dataset_from_data_files(
    annotations_data_directory=source_annotations_directory,
    sequence_data_directory=sequence_data_directory,
    limit=None,
)

formatted_dataset = db.format_dataset(
    dataset=dataset,
    pool_frame_data_by_clip=True,
    decimal_precision=4,
    include_unlabeled_data=True,
    include_angles=True,
    include_distances=True,
    include_normalized=True,
    segmentation_strategy="split_on_label",
    segmentation_splitter_label="step_type",
    segmentation_window=10,
    segmentation_window_label="weight_transfer_type",
)

db.write_dataset_to_csv(
    csv_location=merged_annotation_output_directory,
    formatted_dataset=formatted_dataset,
    filename="pooled_angles_distances_last_10_frames"
)

### Flatten on a 10 frame window based on complete training examples (the end of the example will flatten the previous 10 frames into a training row0)

In [ ]:
db = data_builder.BuildAndFormatDatasetJob()
dataset = db.build_dataset_from_data_files(
    annotations_data_directory=source_annotations_directory,
    sequence_data_directory=sequence_data_directory,
    limit=None,
)

formatted_dataset = db.format_dataset(
    dataset=dataset,
    pool_frame_data_by_clip=False,
    decimal_precision=4,
    include_unlabeled_data=True,
    include_angles=True,
    include_distances=True,
    include_normalized=True,
    segmentation_strategy="flatten_on_example",
    segmentation_splitter_label="step_type",
    segmentation_window=10,
    segmentation_window_label="weight_transfer_type",
)

db.write_dataset_to_csv(
    csv_location=merged_annotation_output_directory,
    formatted_dataset=formatted_dataset,
    filename="flatten_on_example_10_frames_2"
)

### Flatten on a 25 frame window based on complete training examples (the end of the example will flatten the previous 25 frames into a training row)

In [ ]:
db = data_builder.BuildAndFormatDatasetJob()
dataset = db.build_dataset_from_data_files(
    annotations_data_directory=source_annotations_directory,
    sequence_data_directory=sequence_data_directory,
    limit=None,
)

formatted_dataset = db.format_dataset(
    dataset=dataset,
    pool_frame_data_by_clip=False,
    decimal_precision=4,
    include_unlabeled_data=True,
    include_angles=True,
    include_distances=True,
    include_normalized=True,
    segmentation_strategy="flatten_on_example",
    segmentation_splitter_label="step_type",
    segmentation_window=25,
    segmentation_window_label="weight_transfer_type",
)

db.write_dataset_to_csv(
    csv_location=merged_annotation_output_directory,
    formatted_dataset=formatted_dataset,
    filename="preprocessed_flatten_on_example_25_frames_2"
)

### Dataset with all frames as rows

In [ ]:
db = data_builder.BuildAndFormatDatasetJob()
dataset = db.build_dataset_from_data_files(
    annotations_data_directory=source_annotations_directory,
    sequence_data_directory=sequence_data_directory,
    limit=None,
)

formatted_dataset = db.format_dataset(
    dataset=dataset,
    pool_frame_data_by_clip=False,
    decimal_precision=4,
    include_unlabeled_data=True,
    include_angles=True,
    include_distances=True,
    include_normalized=True,
    segmentation_strategy="none",
    segmentation_splitter_label="step_type",
    segmentation_window=25,
    segmentation_window_label="weight_transfer_type",
)

db.write_dataset_to_csv(
    csv_location=merged_annotation_output_directory,
    formatted_dataset=formatted_dataset,
    filename="preprocessed_all_rows.csv",
)